In [ ]:
import nltk
import json
import random
import pickle
from keras.models import load_model
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [ ]:
nltk.download('wordnet')
nltk.download('punkt_tab')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
model = load_model('/content/chatbot_model.h5')
words = pickle.load(open('/content/words.pkl','rb'))
classes = pickle.load(open('/content/classes.pkl','rb'))

In [ ]:
with open('/content/intents.json') as data_file:
    intents = json.load(data_file)

In [ ]:
def clean_up_sentence(sentence):
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word - create short form for word
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

In [ ]:
import numpy as np
def bow(sentence, words, show_details=True):
  sentence_words = clean_up_sentence(sentence)
  bag = [0]*len(words)

  for s in sentence_words:
    for i,w in enumerate(words):
      if w==s:
        bag[i]=1
        if show_details:
          print ("found in bag: %s" % w)

  return np.array(bag)

In [ ]:
def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
      return_list.append({'intent': classes[r[0]], "probability": str(r[1])})
    return return_list

In [ ]:
def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

In [ ]:
def chatbot_response(text):
    ints = predict_class(text, model)
    res = getResponse(ints, intents)
    return res

In [ ]:
import ipywidgets as widgets
from IPython.display import display

# Function to simulate sending a message
def send(change):
    msg = EntryBox.value.strip()
    EntryBox.value = ""  # Clear the input box

    if msg != '':
        # Display the user message
        print(f"You: {msg}")
        res = chatbot_response(msg)
        print(f"Bot: {res}")

# Create a Text widget to enter the message
EntryBox = widgets.Textarea(placeholder='Type your message', description='Message:', disabled=False)

# Create a Button widget to send the message
SendButton = widgets.Button(description="Send")

# Display the widgets
display(EntryBox, SendButton)

# Set up the button click event
SendButton.on_click(send)

Textarea(value='', description='Message:', placeholder='Type your message')

Button(description='Send', style=ButtonStyle())

You: hi
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
Bot: Good to see you again
You: what can you do for me
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
Bot: Offering support for Adverse drug reaction, Blood pressure, Hospitals and Pharmacies
You: Tell me details about blood pressure
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
Bot: Navigating to Blood Pressure module
You: .
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
Bot: Hi there, how can I help?
You: I want to search for blood pressure result history
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
Bot: Please provide Patient ID
You: 1234556
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
Bot: Hi there, how can I help?
